# Etsy API - Analyzing popular printed fabrics

In [23]:
import os
from dotenv import load_dotenv
import pandas as pd
import requests
import json
import plotly.express as px

In [5]:
load_dotenv()
KEY = os.getenv("KEYSTRING")
SECRET = os.getenv("SECRET")

## Get all listings for a specific store

In [26]:
def get_store(name):
    r = dict(requests.get(f'https://openapi.etsy.com/v2/shops/{name}/listings/active?api_key={KEY}').json())
    df = pd.DataFrame(r['results'])
    return df

## Visualize the views/number of favorites for a specific store

In [85]:
def viz_favorites(DF, store=None, terms=None):
    if store:
        title = f'Views / Favorites for Store {store}'
    elif terms:
        title = f'Views / Favorites for Terms: {terms}'
    else: title = ''
    fig = px.scatter(DF, x='views',y='num_favorers', hover_name='title', title=title)
    return fig

In [86]:
viz_favorites(get_store('zeetzeet'), store='zeetzeet')

## Get all listings with terms containing keywords

In [87]:
def get_listings(keywords):
    r = requests.get(f"https://openapi.etsy.com/v2/listings/active.js?tags={'+'.join(keywords)}&api_key={KEY}")
    d = dict(json.loads(r.text[5:-2]))
    print(f"Found {d['count']} listings")
    df = pd.DataFrame(d['results'])
    return df

In [89]:
AH1 = get_listings(['Alexander Henry','fabric'])

Found 4029 listings


In [90]:
viz_favorites(AH1, terms=['Alexander Henry','fabric'])

In [91]:
AH2 = get_listings(['Alexander Henry'])

Found 7340 listings


In [92]:
viz_favorites(AH2, terms=['Alexander Henry'])